In [3]:
import pandas as pd
import re
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import ToktokTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
import nltk 
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/eduardo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/eduardo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
data2 = pd.read_csv("data_listo_mining.csv")

In [5]:
data2.head()

,Unnamed: 0,DENOMINACION_AF,CODIGO_FP,D_CONTENIDOS,D_OBJETIVOS,CODIGO_AREA_TRAD
0,0,FORMACION MODIFICACION DEL RCF,Formación complementaria,"1. Establecimiento, operación y restablecimien...",Dar a conocer el contenido más significativo d...,Competencia clave
1,1,INGLÉS TÉCNICO-MARÍTMO DE ESTIBA,Formación complementaria,- Conceptos elementales de gramática. - Salud...,- Difundir la lengua inglesa como herramienta ...,Lenguas extranjeras
2,2,CAP FORMACION CONTINUA MERCANCIAS / VIAJEROS,Transporte y mantenimiento de vehículos,BLOQUE I: CONDUCCION RACIONAL BASADA EN LAS NO...,Conocer las características de la cadena cinem...,Conducción de vehículos por carretera
3,3,ALERGIAS E INTOLERANCIAS ALIMENTARIAS,Industrias alimentarias,TEMA 1. ALERGIAS E INTOLERANCIAS ALIMENTARIAS ...,La creciente preocupación de la sociedad y de ...,Alimentos diversos
4,4,EXECUTIVE HOUSEKEEPING MANAGER,Hostelería y turismo,Modulo I: REGIDURIA DE PISOS. GOBERNANTA DE HO...,Dar toda la formación y habilidades necesarias...,Alojamiento


In [6]:
data2.shape

(1954, 6)

In [7]:
data2['DENOMINACION_AF'].str.lower()

0                          formacion modificacion del rcf
1                        inglés técnico-marítmo de estiba
2            cap formacion continua mercancias / viajeros
3                   alergias e intolerancias alimentarias
4                          executive housekeeping manager
5                      prl para operadores en puente-grua
6                                               word 2013
7                                 word 2013 inicial medio
8                                   prl primeros auxilios
9       implantación de nuevas metodologías de enseñan...
10                          alemán a1.1 fundación linguam
11      analisis económico financiero de la empresa, f...
12      prevención de riesgos en el sector de la const...
13                     planes de evacuación y emergencias
14      preparación certificate in advanced english (cae)
15                            gruas transtainer practicas
16                                                      i
17      direcc

In [8]:
data2.drop('Unnamed: 0', axis=1, inplace=True)

In [9]:
data2.shape

(1954, 5)

In [10]:
from langdetect import detect

data2['Languagereveiw'] = data2['D_CONTENIDOS'].apply(detect)

In [11]:
data2 = data2[data2['Languagereveiw'] != 'en']

In [12]:
data2.shape

(1593, 6)

# Texts  ==>  Stop words removal ==> Punctuation free ==> Word Lemmatization ==> Digit removal ==> Feature Extraction (Tf-Idf) ==> Model training


In [19]:
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import ToktokTokenizer
from nltk.stem.snowball import SnowballStemmer

import nltk 
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = stopwords.words('spanish')

lemma=WordNetLemmatizer()
token=ToktokTokenizer() #EN ppio deberia funcionar con castellano
stemmer = SnowballStemmer('spanish', ignore_stopwords=True)

#tokenizer = nltk.data.load('tokenizers/punkt/spanish.pickle')
def clean_up(text): 
    '''
    # clean all but alpha or ' '
    st="1234567890-=~@#$%^&*()_+[!{;”:\’><.,/?”}]"
    for w in text:
        if w in st:
            text=text.replace(w,"")
            '''
    return ''.join(ch for ch in text if ch.isalpha() or ch == ' ') # [a-zA-Z\s]

def tokenizar(text): 
    # string and returns list: split by ' ', '\n'
    word=token.tokenize(text)
    return (word)

def stem_and_lemmanize(word): #Probar es-lemmatize y SpaCy
    listLemma = []
    stem = [stemmer.stem(word) for word in word]
    for w in stem:
        x=lemma.lemmatize(w)
        listLemma.append(x)
    return " ".join(listLemma).lower()

def only_stem(word): 
    stem = [stemmer.stem(word) for word in word]
    return " ".join(stem).lower()

def stopWordsRemove(text):
    text = ' '.join([word for word in text.split() if word not in stopwords.words('spanish')])
    #stop = set(stopwords.words('spanish'))
    #wordList =[x.lower().strip() for x in text]
    #removedList=[x for x in wordList if x not in stop]
    #text=" ".join(removedList)
    return text



def get_words(text): 
    limpio = clean_up(text)
    lista = tokenizar(limpio)
    lista2 = only_stem(lista)
    res = stopWordsRemove(lista2)
    return res



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/eduardo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/eduardo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
st = 'hola esto es un curso: de estos!!! de machine learning, el curso se da en: español e inglés. como estamos en España,  1 molamos # con los cursos que cursamos. Trabajo, trabajamos, trabajadores'

st=clean_up(st)
st

'hola esto es un curso de estos de machine learning el curso se da en español e inglés como estamos en España   molamos  con los cursos que cursamos Trabajo trabajamos trabajadores'

In [21]:
st1=tokenizar(st)
st1

['hola',
 'esto',
 'es',
 'un',
 'curso',
 'de',
 'estos',
 'de',
 'machine',
 'learning',
 'el',
 'curso',
 'se',
 'da',
 'en',
 'español',
 'e',
 'inglés',
 'como',
 'estamos',
 'en',
 'España',
 'molamos',
 'con',
 'los',
 'cursos',
 'que',
 'cursamos',
 'Trabajo',
 'trabajamos',
 'trabajadores']

In [22]:
st2 = stem_and_lemmanize(st1)
st2

'hol esto e un cur de estos de machin learning el cur se da en español e ingles como estamos en españ mol con los cur que cur trabaj trabaj trabaj'

In [23]:
stopWordsRemove(st2)

'hol cur machin learning cur da español ingles españ mol cur cur trabaj trabaj trabaj'

In [24]:
get_words(st)
'hol curs machin learning curs da español ingles españ mol curs curs trabaj trabaj trabaj'

'hol curs machin learning curs da español ingles españ mol curs curs trabaj trabaj trabaj'

In [25]:
cosa = 'hola esto es un curso: de estos!!! de machine learning, el curso se da en: español e inglés. como estamos en España,  1 molamos # con los cursos que cursamos. Trabajo, trabajamos, trabajadores'

In [26]:
cosa1 = clean_up(cosa)
cosa1

'hola esto es un curso de estos de machine learning el curso se da en español e inglés como estamos en España   molamos  con los cursos que cursamos Trabajo trabajamos trabajadores'

In [27]:
cosa2 = tokenizar(cosa1)
cosa2

['hola',
 'esto',
 'es',
 'un',
 'curso',
 'de',
 'estos',
 'de',
 'machine',
 'learning',
 'el',
 'curso',
 'se',
 'da',
 'en',
 'español',
 'e',
 'inglés',
 'como',
 'estamos',
 'en',
 'España',
 'molamos',
 'con',
 'los',
 'cursos',
 'que',
 'cursamos',
 'Trabajo',
 'trabajamos',
 'trabajadores']

In [30]:
cosa3 = only_stem(cosa2)
cosa3

'hol esto es un curs de estos de machin learning el curs se da en español e ingles como estamos en españ mol con los curs que curs trabaj trabaj trabaj'

In [31]:
cosa4 = stopWordsRemove(cosa3)
cosa4

'hol curs machin learning curs da español ingles españ mol curs curs trabaj trabaj trabaj'

In [33]:
data2['D_CONTENIDOS']=data2['D_CONTENIDOS'].apply(get_words)

#get_words

In [34]:
data2.head()

,DENOMINACION_AF,CODIGO_FP,D_CONTENIDOS,D_OBJETIVOS,CODIGO_AREA_TRAD,Languagereveiw
0,FORMACION MODIFICACION DEL RCF,Formación complementaria,establec oper restablec bloque telefon situaci...,Dar a conocer el contenido más significativo d...,Competencia clave,es
1,INGLÉS TÉCNICO-MARÍTMO DE ESTIBA,Formación complementaria,concept elemental gramat salud expresion socia...,- Difundir la lengua inglesa como herramienta ...,Lenguas extranjeras,es
2,CAP FORMACION CONTINUA MERCANCIAS / VIAJEROS,Transporte y mantenimiento de vehículos,bloqu i conduccion racional bas norm segur con...,Conocer las características de la cadena cinem...,Conducción de vehículos por carretera,es
3,ALERGIAS E INTOLERANCIAS ALIMENTARIAS,Industrias alimentarias,tem alergi intoler alimentari alergi intoler c...,La creciente preocupación de la sociedad y de ...,Alimentos diversos,es
4,EXECUTIVE HOUSEKEEPING MANAGER,Hostelería y turismo,modul i regiduri pis gobernant hotel hras modu...,Dar toda la formación y habilidades necesarias...,Alojamiento,pt


In [35]:
#stop = set(stopwords.words('spanish'))
#exclude = set(string.punctuation)
#lemma = WordNetLemmatizer()
 
# Cleaning the text sentences so that punctuation marks, stop words & digits are removed
#def clean(doc):
    #stop_free = " ".join([i for i in doc.lower().split() if i not in stop])

In [36]:
def purga_numeros_caracteres1(texto):
    texto = texto.lower()
    texto = re.sub('\W', ' ', texto)
    texto = re.sub('\s+', ' ', texto)
    texto = texto.strip(' ')
    return texto

In [37]:
''''columns = ['DENOMINACION_AF','CODIGO_FP', 'D_CONTENIDOS', 'D_OBJETIVOS', 'CODIGO_AREA_TRAD']
for colum in columns:
    data2[colum]=data2[colum].apply(purga_numeros_caracteres1)'''

"'columns = ['DENOMINACION_AF','CODIGO_FP', 'D_CONTENIDOS', 'D_OBJETIVOS', 'CODIGO_AREA_TRAD']\nfor colum in columns:\n    data2[colum]=data2[colum].apply(purga_numeros_caracteres1)"

In [38]:
data2['D_CONTENIDOS'][0]

'establec oper restablec bloque telefon situacion anormal banaliz temporal via entreg via bloqu coordin respons circul puntualiz prescripcion circul situacion degrad consign libr telefonem'

In [39]:
data2['D_CONTENIDOS'][100]

'practic oral intens estructur vocabulari ingles expresion social expres opinion ofert invit peticion cortes funcion tipic viaj reunion trabaj congres etc'

In [40]:
''''from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
import nltk 
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re
import numpy as np
from collections import Counter''''''

 
stop = set(stopwords.words('spanish'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
 
# Cleaning the text sentences so that punctuation marks, stop words & digits are removed
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    processed = re.sub(r"\d+","",normalized)
    y = processed.split()
    return y'''''
    

     	

'\'from sklearn.feature_extraction.text import TfidfVectorizer\nfrom sklearn.neighbors import KNeighborsClassifier\nfrom sklearn.cluster import KMeans\nimport nltk \nnltk.download(\'stopwords\')\nnltk.download(\'wordnet\')\nfrom nltk.corpus import stopwords\nfrom nltk.stem.wordnet import WordNetLemmatizer\nimport string\nimport re\nimport numpy as np\nfrom collections import Counter\n\n \nstop = set(stopwords.words(\'spanish\'))\nexclude = set(string.punctuation)\nlemma = WordNetLemmatizer()\n \n# Cleaning the text sentences so that punctuation marks, stop words & digits are removed\ndef clean(doc):\n    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])\n    punc_free = \'\'.join(ch for ch in stop_free if ch not in exclude)\n    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())\n    processed = re.sub(r"\\d+","",normalized)\n    y = processed.split()\n    return y'

In [41]:
data2.to_csv('preparado_para_algoritmo.csv',index=False)

In [42]:
'establecimiento operación restablecimiento bloqueo telefónico situacion anormalidad banalización tempor vía entrega vía bloqueada coordinación entr respons circulación puntualizacion sobr prescripcion circulación situacion degradadas consigna libro telefonema'

'establecimiento operación restablecimiento bloqueo telefónico situacion anormalidad banalización tempor vía entrega vía bloqueada coordinación entr respons circulación puntualizacion sobr prescripcion circulación situacion degradadas consigna libro telefonema'